In [24]:
import os
import sys
import torch
import importlib
sys.path.append(os.path.abspath('/home/lh3288/project/PJ2504_Dual_stream/LLM_Dual_Stream/LLM_Probing/code'))
print(sys.executable)
work_dir = '/home/lh3288/project/PJ2504_Dual_stream/LLM_Dual_Stream/LLM_Probing'
os.chdir(work_dir)

cache_dir = '/home/lh3288/.cache/huggingface/hub'

from config import ExperimentConfig
from model_loader import ModelLoader
import neuroling_probing
torch.cuda.empty_cache()
print(f"Current GPU memory allocated: {torch.cuda.memory_allocated() / 1024**3:.2f} GB")
print(f"Current GPU memory cached: {torch.cuda.memory_reserved() / 1024**3:.2f} GB")
!nvidia-smi
!pwd

In [3]:
!pwd

In [25]:
import config
importlib.reload(config)
from config import ExperimentConfig

torch.cuda.empty_cache()
print(f"Current GPU memory allocated: {torch.cuda.memory_allocated() / 1024**3:.2f} GB")
print(f"Current GPU memory cached: {torch.cuda.memory_reserved() / 1024**3:.2f} GB")

config = ExperimentConfig(
    batch_size=32,
    sent_pooling='last',
    output_hidden_states=True,
    speech_mode=False,
    device='cuda' if torch.cuda.is_available() else 'cpu',
    # pca_dim=500,
    test_mode=False,
    task='blimp', # syntax
    # embed_path=None,
    mat_path='/engram/naplab/users/lh3288/wugs/comps_wugs.mat',
    gpu_classify=False,
    lang='en',
    residual_mode=True,
    # save_linear_weights=True,
    # bow_mode=True,
    # save_to_mat=True,
    filter_mode=True,
    blimp_residual_filter=True
)

date = '20250604'
# ling_field = 'semantics_syntax'

hf_token = os.environ["HF_TOKEN"]
gpu_count = 1
use_gpu = True
task = config.task
lang_list = ['en']
# model_list = ['Qwen/Qwen2.5-32B']
model_list = ['Qwen/Qwen2.5-14B']
# model_list = ['Qwen/Qwen2.5-7B']

model_short_list = ['qwen2.5_14B']
config

### single models

In [3]:
i = 0
model_name = model_list[i]
model_name_short = model_short_list[i]

loader = ModelLoader(
        model_name=model_name,
        cache_dir=cache_dir,
        hf_token=hf_token,
        use_gpu=use_gpu,
        gpu_count=gpu_count,
        speech_mode=False,
        baseline_mode=False,
        dtype=torch.bfloat16,
        # untrained=True
    )

model, tokenizer = loader.load()

In [26]:
i = 0
model_name = model_list[i]
model_name_short = model_short_list[i]
model, tokenizer = None, None

In [ ]:
i = 0
model_name = 'bow'
model_name_short = 'bow'
model, tokenizer = None, None  

In [5]:
# print tokenizer's pad token and eos token
print(tokenizer.pad_token_id, tokenizer.eos_token_id)
print(tokenizer.pad_token, tokenizer.eos_token)

In [ ]:
# empty cuda cache
# del model, tokenizer
torch.cuda.empty_cache()
# collect garbage
import gc
gc.collect()
torch.cuda.empty_cache()

In [1]:
!nvidia-smi

In [27]:
importlib.reload(neuroling_probing)

level = "base"
# level = "wugs_dist"
suffix = "_scale"
# suffix = ""
output_file_name = work_dir+f"/results/{task}/{task}_{level}_{model_name_short}_{str(config.pca_dim)}{suffix}.csv"
print("Output file:", output_file_name)

config.update({
    'device': torch.device('cuda' if torch.cuda.is_available() and use_gpu else 'cpu'),
    'output_file_name': output_file_name,
    'model_name': model_name_short
})

task_runner = neuroling_probing.NeurolingProbing(config, model, tokenizer=tokenizer)

if task == 'blimp':
    task_runner.run_blimp('./data/BLiMP/')
elif task == 'comps':
    config.update({'lang': 'en'})
    task_runner.config = config
    task_runner.run_comps(work_dir+f'/data/COMPS/comps_{level}.jsonl')
elif task == 'comps_wugs':
    task_runner.run_comps(work_dir+'/data/COMPS/')
elif task == "proofwriter":
    task_runner.run_proofwriter(os.path.join(work_dir, 'data/ProofWriter/proofwriter.jsonl'))
elif task == "logic-llm":
    task_runner.run_proofwriter(os.path.join(work_dir, 'data/Logic-LLM/logic_llm.jsonl'))
elif task == "prontoqa":
    task_runner.run_prontoqa(os.path.join(work_dir, 'data/ProntoQA/prontoqa.jsonl'))   
else:
    print('task not supported')

torch.cuda.empty_cache()

print("All done.")

In [17]:
!pwd

In [40]:
import cupy as cp
from cuml import PCA

X = cp.random.rand(10000, 4098).astype(cp.float32)
pca = PCA(n_components=1024)
X_transformed = pca.fit_transform(X)
print("PCA finished")

In [1]:
!nvidia-smi

In [18]:
import pandas as pd
df_before = pd.read_json(work_dir+'/data/COMPS/'+'comps_wugs_dist-before.jsonl', lines=True)

In [21]:
df_before_task = df_before[df_before['negative_sample_type'] == 'taxonomic']

In [22]:
sentences_task_good = df_before_task['prefix_acceptable'] + ' ' + df_before_task['property_phrase']

In [25]:
len(df_before)

In [19]:
df_before[:10]